artist.json.gzは，オープンな音楽データベースMusicBrainzの中で，アーティストに関するものをJSON形式に変換し，gzip形式で圧縮したファイルである．

このファイルには，1アーティストに関する情報が1行にJSON形式で格納されている．

JSON形式の概要は以下の通りである．

artist.json.gzのデータをKey-Value-Store (KVS) およびドキュメント志向型データベースに格納・検索することを考える．

KVSとしては，LevelDB，Redis，KyotoCabinet等を用いよ．

ドキュメント志向型データベースとして，MongoDBを採用したが，CouchDBやRethinkDB等を用いてもよい．

** 0. KVSの構築 **

Key-Value-Store (KVS) を用い，アーティスト名（name）から活動場所（area）を検索するためのデータベースを構築せよ．

In [1]:
import gzip
import json

*** leveldbパッケージを使った場合 ***

In [2]:
fname = 'nlp100data/artist.json.gz'
fname_db = 'nlp100data/test_db'

In [3]:
import leveldb

In [4]:
db = leveldb.LevelDB(fname_db)

In [5]:
with gzip.open(fname, 'rt') as data_file:
    for i , line in enumerate(data_file):
        if i > -1:
            run_dict = json.loads(line)
            key = "{}\t{}".format(run_dict['name'] , run_dict['id'])
            #print(key)
            value = run_dict.get("area" , "")
            db.Put(key.encode() , value.encode())

            #print("[{}] {}".format(value ,key))

In [7]:
Ndata = len(list(db.RangeIter(include_value=False)))

In [8]:
print("registered {} pairs".format(Ndata))

registered 921337 pairs


**** sandbox ****


In [6]:
run_dict

{'aliases': [{'name': 'Ranks, Shabba', 'sort_name': 'Ranks, Shabba'},
  {'name': 'Shaba Ranks', 'sort_name': 'Shaba Ranks'},
  {'name': 'Snabba Ranks', 'sort_name': 'Snabba Ranks'},
  {'name': 'Shabba', 'sort_name': 'Shabba'}],
 'area': 'Jamaica',
 'begin': {'date': 17, 'month': 1, 'year': 1966},
 'ended': True,
 'gender': 'Male',
 'gid': 'c0336f2a-610f-4b38-bc13-eb07393425a7',
 'id': 37082,
 'name': 'Shabba Ranks',
 'sort_name': 'Ranks, Shabba',
 'tags': [{'count': 1, 'value': 'raggamuffin'},
  {'count': 1, 'value': 'dancehall'},
  {'count': 1, 'value': 'pop'},
  {'count': 1, 'value': 'pop and chart'},
  {'count': 1, 'value': 'drum and bass'},
  {'count': 1, 'value': 'jungle'},
  {'count': 1, 'value': 'mc'}],
 'type': 'Person'}

In [11]:
%%bash
pwd

/home/toshinao/PycharmProjects/keras_sandbox


In [17]:
import codecs

tmp = list()
with codecs.open('nlp100data/tmp' , 'r' , 'utf-8') as rf:
    for l in rf:
        tmp.append(l)

In [16]:
tmp[0]

'{"name": "WIK▲N", "tags": [{"count": 1, "value": "sillyname"}], "sort_name": "WIK▲N", "ended": true, "gid": "8972b1c1-6482-4750-b51f-596d2edea8b1", "id": 805192}\n'


*** plyvelパッケージを使った場合 (暫定的にこちらをメインに)***

https://qiita.com/tomotaka_ito/items/60c65dd5261fdfc6e71a

https://plyvel.readthedocs.io/en/latest/user.html#getting-started

http://kuroneko0208.hatenablog.com/entry/2014/10/06/090128

In [2]:
fname = 'nlp100data/artist.json.gz'
fname_db = 'nlp100data/test2_db'

In [3]:
import plyvel

In [4]:
plyvel.DB?

In [5]:
db = plyvel.DB(fname_db , create_if_missing=True)

In [6]:
with gzip.open(fname, 'rt') as data_file:
    for i , line in enumerate(data_file):
        if i > -1:
            run_dict = json.loads(line)
            #key = "{}\t{}".format(run_dict['name'] , run_dict['id'])
            key = run_dict['name']
            #print(key)
            value = run_dict.get("area" , "")
            if value != "":
                db.put(key.encode() , value.encode())

            #print("[{}] {}".format(value ,key))

In [7]:
len([key for key , value in db])

309317

In [8]:
db.close()

#### KVSの検索

60で構築したデータベースを用い，特定の（指定された）アーティストの活動場所を取得せよ．

In [9]:
db = plyvel.DB(fname_db , create_if_missing=True)

In [25]:
artist_name = "10 Rue d'la Madeleine"

In [26]:
db.get(artist_name.encode()).decode()

'France'

##### バイト列型に関するメモ

- encodeでバイト列型に
- decodeで例えばUTF-8に

##### sandbox

In [16]:
tmp = [key_value[0].decode()  for i , key_value in enumerate(db) if i < 300 and i > 290]

In [17]:
tmp

['1/3 Octave Band',
 '10 5 Neuton',
 '10 Fold B-Low',
 '10 Ft. Ganja Plant',
 '10 Minute Warning',
 "10 Rue d'la Madeleine",
 '10 Years',
 '10 petits indiens',
 '10" Maria']

In [24]:
db.get("10 Rue d'la Madeleine".encode()).decode()

'France'

In [35]:
db.get('"Pretty" Fedd'.encode())

b''

In [32]:
["{} [{}]".format(key_value[0].decode() , key_value[1].decode()) for i , key_value in enumerate(db) if i < 100 and i > 90]

['"Pretty" Fedd []',
 '"Red" Roberts mit seinem Ultraphon Jazz-Orchester [Germany]',
 '"Ritekt Jerk", Mases Erik Jonsson []',
 '"Rob,SEAL" []',
 '"Rolling Joe" Johnson []',
 '"Rural" Merle Hicks []',
 '"See There" Singing Band []',
 '"Sir" Oliver Mally [Austria]',
 '"Slim" Jim Smith []']

In [12]:
sample_names = [key_value[0] for i , key_value in enumerate(db) if i < 100]

In [13]:
sample_names[70].decode()

"'Jabbo' Williams"

In [15]:
db.get(sample_names[7]).decode()

'Sankt-Peterburg'

In [25]:
db.get(b"!ATTENTION!\t617688")

b''

In [26]:
db.get(b'!Action Pact!\\t95246')

In [28]:
db.get(b'!Bang Elektronika\\t172294')

In [41]:
%%bash
git commit -a -m "60"

[master 6618d1c] 60
 1 file changed, 128 insertions(+), 8 deletions(-)


#### KVS内の反復処理
60で構築したデータベースを用い，活動場所が「Japan」となっているアーティスト数を求めよ．

In [32]:

run_place = "Japan"

In [33]:
run_place.encode()

b'Japan'

In [42]:
japan_artists = [x[0].decode() for x in db.iterator() if x[1] == run_place.encode()]

In [44]:
print("Number of registered artists in Japan is {}".format(len(japan_artists)))

Number of registered artists in Japan is 22128


In [49]:
japan_artists[100:120]

['6号さん',
 '7!!',
 '72',
 '742',
 '765PRO ALLSTARS',
 '7@',
 '7chi子♪',
 '7人祭',
 '800 Cherries',
 '876PRO ALLSTARS',
 '96',
 '96ちゃん',
 '98',
 '99RadioService',
 '9GOATS BLACK OUT',
 '9mm Parabellum Bullet',
 '9nine',
 '9少女',
 '>>96',
 'A Lunch']

####  オブジェクトを値に格納したKVS
KVSを用い，アーティスト名（name）からタグと被タグ数（タグ付けされた回数）のリストを検索するためのデータベースを構築せよ．さらに，ここで構築したデータベースを用い，アーティスト名からタグと被タグ数を検索せよ．

In [52]:
fname_db63 = 'nlp100data/test63_db'

In [53]:
db = plyvel.DB(fname_db63 , create_if_missing=True)

In [57]:
with gzip.open(fname, 'rt') as data_file:
    for i , line in enumerate(data_file):
        if i > -1:
            run_dict = json.loads(line)
            #key = "{}\t{}".format(run_dict['name'] , run_dict['id'])
            key = run_dict['name']
            #print(key)
            value = run_dict.get("tags" , "")
            if value != "":
                db.put(key.encode() , json.dumps(value).encode())

            #print("[{}] {}".format(value ,key))

In [61]:
artist_name = "Oasis"

In [65]:
run_tags = json.loads(db.get(artist_name.encode()).decode())

In [69]:
for x in run_tags:
    print("{} ({})".format(x.get("value") , x.get("count")))
    

rock (1)
britpop (3)
british (4)
uk (1)
britannique (1)
rock and indie (1)
england (1)
manchester (1)


##### jsonに関するメモ

###### json.dumpsでjson化

In [56]:
json.dumps(value)

'[{"value": "sillyname", "count": 1}]'

###### json.loadsでdict化（dumpsの逆）

In [60]:
json.loads(json.dumps(run_dict))

{'aliases': [{'name': 'Ranks, Shabba', 'sort_name': 'Ranks, Shabba'},
  {'name': 'Shaba Ranks', 'sort_name': 'Shaba Ranks'},
  {'name': 'Snabba Ranks', 'sort_name': 'Snabba Ranks'},
  {'name': 'Shabba', 'sort_name': 'Shabba'}],
 'area': 'Jamaica',
 'begin': {'date': 17, 'month': 1, 'year': 1966},
 'ended': True,
 'gender': 'Male',
 'gid': 'c0336f2a-610f-4b38-bc13-eb07393425a7',
 'id': 37082,
 'name': 'Shabba Ranks',
 'sort_name': 'Ranks, Shabba',
 'tags': [{'count': 1, 'value': 'raggamuffin'},
  {'count': 1, 'value': 'dancehall'},
  {'count': 1, 'value': 'pop'},
  {'count': 1, 'value': 'pop and chart'},
  {'count': 1, 'value': 'drum and bass'},
  {'count': 1, 'value': 'jungle'},
  {'count': 1, 'value': 'mc'}],
 'type': 'Person'}

##### sandbox

In [84]:
%%bash
git commit -a -m'from notebook'

[master e963fe4] from notebook
 1 file changed, 109 insertions(+), 3 deletions(-)


In [67]:
[x for x in run_tags][0]

{'count': 1, 'value': 'rock'}

In [70]:
db.iterator?

In [72]:

db.get?

In [73]:
tmp = [x for x in db.iterator() if x[0] == artist_name.encode()]

In [76]:
len(tmp)


1

####  MongoDBの構築

アーティスト情報（artist.json.gz）をデータベースに登録せよ．

さらに，次のフィールドでインデックスを作成せよ: 

name, aliases.name, tags.value, rating.value

##### 起動まで

Ubuntuの場合、

https://docs.mongodb.com/manual/tutorial/install-mongodb-on-ubuntu

に従えば良い

起動は以下：

In [ ]:
sudo service mongod start

##### 課題

https://qiita.com/yuji0602/items/c55e2cb75376fd565b4e

In [79]:
import pymongo

In [ ]:
from pymongo import MongoClient


unit_bulk = 10000       # バルクインサートする単位（件）

# MongoDBのデータベースtestdbにコレクションartistを作成

In [ ]:
client = MongoClient()

In [ ]:
db = client.testdb　

collectionがRDBにおけるテーブルに相当する模様：

In [ ]:
collection = db.artist　# 

In [80]:
# gzファイル読み込み
with gzip.open(fname, 'rt') as data_file:

    # 1行ずつパースしてbufに詰め込む
    buf = []
    for i, line in enumerate(data_file, 1):
        data_json = json.loads(line)
        buf.append(data_json)

        # unit_bulk件たまったらartistへバルクインサート
        if i % unit_bulk == 0:
            collection.insert_many(buf)
            buf = []
            print('{}件追加完了'.format(i))

10000件追加完了
20000件追加完了
30000件追加完了
40000件追加完了
50000件追加完了
60000件追加完了
70000件追加完了
80000件追加完了
90000件追加完了
100000件追加完了
110000件追加完了
120000件追加完了
130000件追加完了
140000件追加完了
150000件追加完了
160000件追加完了
170000件追加完了
180000件追加完了
190000件追加完了
200000件追加完了
210000件追加完了
220000件追加完了
230000件追加完了
240000件追加完了
250000件追加完了
260000件追加完了
270000件追加完了
280000件追加完了
290000件追加完了
300000件追加完了
310000件追加完了
320000件追加完了
330000件追加完了
340000件追加完了
350000件追加完了
360000件追加完了
370000件追加完了
380000件追加完了
390000件追加完了
400000件追加完了
410000件追加完了
420000件追加完了
430000件追加完了
440000件追加完了
450000件追加完了
460000件追加完了
470000件追加完了
480000件追加完了
490000件追加完了
500000件追加完了
510000件追加完了
520000件追加完了
530000件追加完了
540000件追加完了
550000件追加完了
560000件追加完了
570000件追加完了
580000件追加完了
590000件追加完了
600000件追加完了
610000件追加完了
620000件追加完了
630000件追加完了
640000件追加完了
650000件追加完了
660000件追加完了
670000件追加完了
680000件追加完了
690000件追加完了
700000件追加完了
710000件追加完了
720000件追加完了
730000件追加完了
740000件追加完了
750000件追加完了
760000件追加完了
770000件追加完了
780000件追加完了
790000件追加完了
800000件追加完了
810000件追加完了
820000件追加完了
830000件追加完了
840000件追加完了
8

 インデックス作成

In [82]:
collection.create_index([('name', pymongo.ASCENDING)])  
collection.create_index([('aliases.name', pymongo.ASCENDING)])  
collection.create_index([('tags.value', pymongo.ASCENDING)])
collection.create_index([('rating.value', pymongo.ASCENDING)])

'rating.value_1'

##### sandbox

In [78]:
%%bash
conda install pymongo

Solving environment: ...working... done

## Package Plan ##

  environment location: /home/toshinao/anaconda3

  added / updated specs: 
    - pymongo


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    pymongo-3.4.0              |           py35_0         412 KB
    conda-4.5.8                |           py35_0         1.0 MB
    ------------------------------------------------------------
                                           Total:         1.4 MB

The following NEW packages will be INSTALLED:

    pymongo: 3.4.0-py35_0

The following packages will be UPDATED:

    conda:   4.5.4-py35_0 --> 4.5.8-py35_0

Proceed ([y]/n)? 

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


pymongo-3.4.0        |  412 KB | ########## | 100% 
conda-4.5.8          |  1.0 MB | ########## | 100% 


#### MongoDBの検索
MongoDBのインタラクティブシェルを用いて，"Queen"というアーティストに関する情報を取得せよ．さらに，これと同様の処理を行うプログラムを実装せよ．


In [86]:
[x for x in collection.find({'name': 'Queen'})]

[{'_id': ObjectId('5b5c4df89fea6f288b71ac5b'),
  'aliases': [{'name': 'Queen', 'sort_name': 'Queen'}],
  'area': 'Japan',
  'ended': True,
  'gender': 'Female',
  'gid': '420ca290-76c5-41af-999e-564d7c71f1a7',
  'id': 701492,
  'name': 'Queen',
  'sort_name': 'Queen',
  'tags': [{'count': 1, 'value': 'kamen rider w'},
   {'count': 1, 'value': 'related-akb48'}],
  'type': 'Character'},
 {'_id': ObjectId('5b5c4df99fea6f288b727307'),
  'aliases': [{'name': '女王', 'sort_name': '女王'}],
  'area': 'United Kingdom',
  'begin': {'date': 27, 'month': 6, 'year': 1970},
  'ended': True,
  'gid': '0383dadf-2a4e-4d10-a46a-e9e041da8eb3',
  'id': 192,
  'name': 'Queen',
  'rating': {'count': 24, 'value': 92},
  'sort_name': 'Queen',
  'tags': [{'count': 2, 'value': 'hard rock'},
   {'count': 1, 'value': '70s'},
   {'count': 1, 'value': 'queen family'},
   {'count': 1, 'value': '90s'},
   {'count': 1, 'value': '80s'},
   {'count': 1, 'value': 'glam rock'},
   {'count': 4, 'value': 'british'},
   {'count

##### son.dumps()でObjectIdを処理するための関数

In [115]:
from bson.objectid import ObjectId
def support_ObjectId(obj):
    '''json.dumps()でObjectIdを処理するための関数
    ObjectIdはjsonエンコードできない型なので、文字列型に変換する

    戻り値：
    ObjectIdから変換した文字列
    '''
    if isinstance(obj, ObjectId):
        return str(obj)     # 文字列として扱う
    raise TypeError(repr(obj) + " is not JSON serializable")

In [116]:
[json.dumps(x , default=support_ObjectId) for x in collection.find({'name': 'Queen'})]

['{"name": "Queen", "aliases": [{"name": "Queen", "sort_name": "Queen"}], "type": "Character", "ended": true, "sort_name": "Queen", "tags": [{"value": "kamen rider w", "count": 1}, {"value": "related-akb48", "count": 1}], "area": "Japan", "id": 701492, "_id": "5b5c4df89fea6f288b71ac5b", "gender": "Female", "gid": "420ca290-76c5-41af-999e-564d7c71f1a7"}',
 '{"rating": {"value": 92, "count": 24}, "aliases": [{"name": "\\u5973\\u738b", "sort_name": "\\u5973\\u738b"}], "sort_name": "Queen", "area": "United Kingdom", "id": 192, "_id": "5b5c4df99fea6f288b727307", "begin": {"year": 1970, "date": 27, "month": 6}, "name": "Queen", "type": "Group", "tags": [{"value": "hard rock", "count": 2}, {"value": "70s", "count": 1}, {"value": "queen family", "count": 1}, {"value": "90s", "count": 1}, {"value": "80s", "count": 1}, {"value": "glam rock", "count": 1}, {"value": "british", "count": 4}, {"value": "english", "count": 1}, {"value": "uk", "count": 2}, {"value": "pop/rock", "count": 1}, {"value": "

#### 検索件数の取得
MongoDBのインタラクティブシェルを用いて，活動場所が「Japan」となっているアーティスト数を求めよ．

In [88]:

len([x for x in collection.find({'area': 'Japan'})])

22742

#### 複数のドキュメントの取得
特定の（指定した）別名を持つアーティストを検索せよ．

In [89]:
alias = "スマップ"

In [95]:
hit7 = [x for x in collection.find({'aliases.name':alias})]

In [101]:
hit7[0]

{'_id': ObjectId('5b5c4df29fea6f288b6cb821'),
 'aliases': [{'name': 'スマップ', 'sort_name': 'スマップ'}],
 'area': 'Japan',
 'begin': {'month': 4, 'year': 1988},
 'ended': True,
 'gid': '6ab7953b-8480-41fe-a9fe-5e220d8e9429',
 'id': 265728,
 'name': 'SMAP',
 'sort_name': 'SMAP',
 'tags': [{'count': 1, 'value': 'likedis auto'}],
 'type': 'Group'}

In [112]:
json.dumps(hit7[0] , default=support_ObjectId ,ensure_ascii=False , sort_keys=True)

'{"_id": "5b5c4df29fea6f288b6cb821", "aliases": [{"name": "スマップ", "sort_name": "スマップ"}], "area": "Japan", "begin": {"month": 4, "year": 1988}, "ended": true, "gid": "6ab7953b-8480-41fe-a9fe-5e220d8e9429", "id": 265728, "name": "SMAP", "sort_name": "SMAP", "tags": [{"count": 1, "value": "likedis auto"}], "type": "Group"}'

##### sandbox

In [118]:
support_ObjectId((hit7[0]['_id']))

'5b5c4df29fea6f288b6cb821'

#### ソート
"dance"というタグを付与されたアーティストの中でレーティングの投票数が多いアーティスト・トップ10を求めよ．

In [119]:
dances = collection.find({'tags.value' : 'dance'})

In [120]:
dances

In [122]:

dances.count()

286

In [124]:
dances.sort('rating.count' , pymongo.DESCENDING)

In [146]:
for i in range(9):
        print("{}({})".format(dances[i]['name'] , dances[i]['rating']['count']))

Madonna(26)
Björk(23)
The Prodigy(23)
Rihanna(15)
Britney Spears(13)
Maroon 5(11)
Adam Lambert(7)
Fatboy Slim(7)
Basement Jaxx(6)


##### sandbox

In [143]:
(dances[0]['tags'][0])

{'count': 1, 'value': 'dance-pop'}

In [145]:
dances[0]['rating']['count']

26

In [141]:

print(json.dumps(dances[0] ,default=support_ObjectId  ,indent = '\t'))

{
	"sort_name": "Madonna",
	"area": "United States",
	"id": 89,
	"_id": "5b5c4dfb9fea6f288b738658",
	"ended": true,
	"rating": {
		"value": 88,
		"count": 26
	},
	"name": "Madonna",
	"type": "Person",
	"tags": [
		{
			"value": "dance-pop",
			"count": 1
		},
		{
			"value": "electropop",
			"count": 1
		},
		{
			"value": "tell me",
			"count": 1
		},
		{
			"value": "pop and chart",
			"count": 1
		},
		{
			"value": "multiple ipi",
			"count": 1
		},
		{
			"value": "electronic",
			"count": 1
		},
		{
			"value": "am\u00e9ricain",
			"count": 1
		},
		{
			"value": "usa",
			"count": 1
		},
		{
			"value": "singer",
			"count": 1
		},
		{
			"value": "chanteur",
			"count": 1
		},
		{
			"value": "american",
			"count": 1
		},
		{
			"value": "pop",
			"count": 4
		},
		{
			"value": "greatest hits",
			"count": 1
		},
		{
			"value": "dance",
			"count": 1
		}
	],
	"gender": "Female",
	"gid": "79239441-bfd5-4981-a70c-55c3f15c1287",
	"begin": {
		"year": 1958,
		"date": 16,
		"mont

#### Webアプリケーションの作成
ユーザから入力された検索条件に合致するアーティストの情報を表示するWebアプリケーションを作成せよ．アーティスト名，アーティストの別名，タグ等で検索条件を指定し，アーティスト情報のリストをレーティングの高い順などで整列して表示せよ．